In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Usage Example
if __name__ == "__main__":
    # Define the path to your YAML config
    config_file = "/storage_common/angiod/nmr/data/atom_type_config.protein+nucleotide.yaml"

    # Initialize the NMRDataset with the config file
    from csnet.utils.dataset import NMRDatasetBuilder
    dataset_builder = NMRDatasetBuilder(config_file)

    data_root = '/storage_common/angiod/nmr/bmrb'

    dataset_builder.build_statistics(data_root=data_root)
    dataset_builder.extract_outliers()

In [ ]:
dataset_builder.outliers

In [ ]:
resnames = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'CYX', 'GLN', 'GLU',
            'GLY', 'HID', 'HIE', 'HIS', 'ILE', 'LEU', 'LYS', 'MET',
            'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
# resnames = ['A', 'G', 'U', 'C']
# Either provide atomnames OR note_types
resnames = ["MET"]
atomnames = ["CE"]
dataset_builder.plot_distribution(resnames, atomnames)

In [114]:
import numpy as np

ds = np.load('/storage_common/angiod/nmr/delete/npz/6E83.30501.npz')

In [ ]:
for k,v in ds.items():
    print(k, v.shape)

In [116]:
af = ds['atom_fullnames']
ar = ds['atom_resnumbers']
cs = ds['chemical_shifts']